In [158]:
# Importing necessary libraries and datasets
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy import sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor

# Merging datasets 

In [159]:

movies_metadata = pd.read_csv('movies_metadata.csv')
credits = pd.read_csv('credits.csv')
ratings_small = pd.read_csv('ratings_small.csv')
links_small = pd.read_csv('links_small.csv')
keywords = pd.read_csv('keywords.csv')


C:\Users\DELL\AppData\Local\Temp\ipykernel_15144\4046238992.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_metadata.csv')


In [160]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [161]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [162]:
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [163]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [164]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [165]:
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
df = movies_metadata.merge(credits, on='id', how='inner')
df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [166]:
keywords['id'] = pd.to_numeric(keywords['id'], errors='coerce')
links_small = links_small[['movieId', 'tmdbId']].dropna()
df = df.merge(keywords, on='id', how='left')
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [167]:
links_small['tmdbId'] = pd.to_numeric(links_small['tmdbId'], downcast='integer', errors='coerce')
df = df.merge(links_small, left_on='id', right_on='tmdbId', how='left')
df = df.merge(ratings_small[['movieId', 'userId', 'rating']], on='movieId', how='left')

In [168]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,cast,crew,keywords,movieId,tmdbId,userId,rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1.0,862.0,7.0,3.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1.0,862.0,9.0,4.0
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1.0,862.0,13.0,5.0
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1.0,862.0,15.0,2.0
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1.0,862.0,19.0,3.0


In [169]:
print(df.shape[0])

137588


## Data Preprocessing

In [170]:
df.isna().sum()

adult                         0
belongs_to_collection    103420
budget                        0
genres                        0
homepage                 103240
id                            0
imdb_id                      17
original_language            11
original_title                0
overview                    997
popularity                    4
poster_path                 402
production_companies          4
production_countries          4
release_date                 88
revenue                       4
runtime                     268
spoken_languages              4
status                       93
tagline                   31685
title                         4
video                         4
vote_average                  4
vote_count                    4
cast                          0
crew                          0
keywords                      0
movieId                   37409
tmdbId                    37409
userId                    37466
rating                    37466
dtype: i

In [171]:
print(df.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'movieId',
       'tmdbId', 'userId', 'rating'],
      dtype='object')


In [172]:
# Handling missing values
df['rating'].fillna(df['rating'].mean(), inplace=True)

df.isna().sum()

adult                         0
belongs_to_collection    103420
budget                        0
genres                        0
homepage                 103240
id                            0
imdb_id                      17
original_language            11
original_title                0
overview                    997
popularity                    4
poster_path                 402
production_companies          4
production_countries          4
release_date                 88
revenue                       4
runtime                     268
spoken_languages              4
status                       93
tagline                   31685
title                         4
video                         4
vote_average                  4
vote_count                    4
cast                          0
crew                          0
keywords                      0
movieId                   37409
tmdbId                    37409
userId                    37466
rating                        0
dtype: i

In [173]:
# Dropping unnecessary columns

df = df[['id', 'movieId', 'userId', 'rating']]
df.isna().sum()

id             0
movieId    37409
userId     37466
rating         0
dtype: int64

In [174]:
# Interpolating missing values
df.interpolate(method='linear', inplace=True)



In [175]:
# Specifying columns for imputation
columns_for_imputation = ['rating', 'userId', 'movieId']

# Initializing KNNImputer & Imputing missing values using KNN
imputer = KNNImputer(n_neighbors=5)
df[columns_for_imputation] = imputer.fit_transform(df[columns_for_imputation])



In [176]:
# Dropping duplicates
df.drop_duplicates(inplace=True)
df.isna().sum()


id         0
movieId    0
userId     0
rating     0
dtype: int64

In [177]:
# # Scaling 'rating' column
# scaler = StandardScaler()
# df['rating_x'] = scaler.fit_transform(df['rating_x'])
# print(df.shape[0])

In [178]:
# Reshaping the 'rating' column to a 2D array
ratings_2d = df['rating'].values.reshape(-1, 1)

# Initializing StandardScaler
scaler = StandardScaler()

# Fitting and transforming the ratings
df['rating'] = scaler.fit_transform(ratings_2d)

# Print the number of rows in the DataFrame
print(df.shape[0])

137276


## Creating Sparse Matrix Representation:
A sparse matrix representation of user-item interactions is created using scipy.sparse.coo_matrix.
This involves mapping unique user and movie IDs to integer indices and constructing a sparse matrix using COO format.

In [179]:
# Creating a mapping of user and movie IDs to integer indices
user_to_index = {user_id: index for index, user_id in enumerate(df['userId'].unique())}
movie_to_index = {movie_id: index for index, movie_id in enumerate(df['movieId'].unique())}

# Creating data arrays for the sparse matrix
user_indices = [user_to_index[user_id] for user_id in df['userId']]
movie_indices = [movie_to_index[movie_id] for movie_id in df['movieId']]
ratings = df['rating']

# Creating the sparse matrix using the data arrays
train_sparse_matrix = sp.coo_matrix((ratings, (user_indices, movie_indices)))

# Converting the COO matrix to CSR format for faster row slicing
train_sparse_matrix = train_sparse_matrix.tocsr()
train_sparse_matrix

<21293x41531 sparse matrix of type '<class 'numpy.float64'>'
	with 132318 stored elements in Compressed Sparse Row format>

In [180]:
# Print the first 5 non-zero elements of the sparse matrix
nonzero_indices = train_sparse_matrix.nonzero()
for i in range(5):
    row_idx = nonzero_indices[0][i]
    col_idx = nonzero_indices[1][i]
    value = train_sparse_matrix[row_idx, col_idx]
    print(f"Value at ({row_idx}, {col_idx}): {value}")

# Print the shape of the sparse matrix
print("Shape of the sparse matrix:", train_sparse_matrix.shape)


Value at (0, 0): -0.6019158373653929
Value at (0, 9): -0.6019158373653929
Value at (0, 20): -0.6019158373653929
Value at (0, 30): -0.6019158373653929
Value at (0, 33): 0.5066254570256833
Shape of the sparse matrix: (21293, 41531)


We created a mapping of unique user IDs and movie IDs to integer indices. This mapping allows us to represent users and movies with integer values, which is necessary for constructing the sparse matrix.

We created data arrays for the sparse matrix. These arrays contain the integer indices corresponding to users, movies, and their ratings.

Using the data arrays, we constructed a sparse matrix using the COO (Coordinate list) format. This sparse matrix represents user-item interactions, where each entry corresponds to a rating given by a user to a movie.

We converted the COO matrix to CSR (Compressed Sparse Row) format for faster row slicing, which is a common operation in recommendation systems.

In [181]:
# Defining utility functions
def get_global_average(sparse_matrix):
    # Calculating the global average rating
    global_average = sparse_matrix.sum() / sparse_matrix.count_nonzero()
    return global_average

def get_average_ratings(sparse_matrix, of_users=True):
    axis = 1 if of_users else 0  # 1 for users, 0 for movies
    sum_of_ratings = sparse_matrix.sum(axis=axis).A1
    is_rated = sparse_matrix != 0
    no_of_ratings = is_rated.sum(axis=axis).A1
    # Calculating average ratings for users or movies
    average_ratings = {i: sum_of_ratings[i] / no_of_ratings[i] for i in range(len(sum_of_ratings)) if no_of_ratings[i] != 0}
    return average_ratings

def get_top_similar_users(train_sparse_matrix, user, movie, train_averages):
    user_sim = cosine_similarity(train_sparse_matrix[user], train_sparse_matrix).ravel()
    top_sim_users = user_sim.argsort()[::-1][1:]  # Ignoring the user itself
    top_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
    top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_users_ratings.extend([train_averages['movie'][movie]] * (5 - len(top_sim_users_ratings)))
    return top_sim_users_ratings

def get_top_similar_movies(train_sparse_matrix, user, movie, train_averages):
    movie_sim = cosine_similarity(train_sparse_matrix[:, movie].T, train_sparse_matrix.T).ravel()
    top_sim_movies = movie_sim.argsort()[::-1][1:]  # Ignoring the movie itself
    top_ratings = train_sparse_matrix[user, top_sim_movies].toarray().ravel()
    top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_movies_ratings.extend([train_averages['user'][user]] * (5 - len(top_sim_movies_ratings)))
    return top_sim_movies_ratings


Utility functions were defined for calculating the global average rating, average ratings for users and movies, and finding top similar users and movies based on cosine similarity.

get_global_average(sparse_matrix):
This function calculates the global average rating from the sparse matrix.
It sums up all the ratings in the matrix and divides by the number of non-zero elements to compute the average rating.
The global average rating is a key metric in recommendation systems and is used for various purposes, such as baseline predictions.

get_average_ratings(sparse_matrix, of_users=True):
This function computes the average ratings for users or movies based on the provided sparse matrix.
Depending on the value of the of_users parameter, it calculates either the average ratings by users (if of_users is True) or by movies (if of_users is False).
It computes the sum of ratings and the count of non-zero ratings along the specified axis (users or movies) and then calculates the average ratings.
The function returns a dictionary where the keys are indices (user or movie indices) and values are the corresponding average ratings.

get_top_similar_users(train_sparse_matrix, user, movie, train_averages):
This function finds the top similar users to a given user based on their ratings for a specific movie.
It computes cosine similarity between the ratings of the given user and all other users using the provided sparse matrix.
The function then identifies the top similar users (excluding the user itself) and retrieves their ratings for the specified movie.
If fewer than 5 similar users are found, it fills the remaining slots with the average rating for that movie.
The function returns a list of top similar users' ratings for the movie.

get_top_similar_movies(train_sparse_matrix, user, movie, train_averages):
This function finds the top similar movies to a given movie based on the ratings provided by the given user.
It computes cosine similarity between the ratings of the specified movie and all other movies using the provided sparse matrix.
The function then identifies the top similar movies (excluding the movie itself) and retrieves the user's ratings for those movies.
If fewer than 5 similar movies are found, it fills the remaining slots with the average rating given by the user.
The function returns a list of top similar movies' ratings provided by the user.

In [182]:
# Calculating global average rating
global_avg_rating = get_global_average(train_sparse_matrix)

# Calculating average ratings for users and movies
user_avg_ratings = get_average_ratings(train_sparse_matrix, of_users=True)
movie_avg_ratings = get_average_ratings(train_sparse_matrix, of_users=False)

# Example usage of top similar users and movies
user_id = 7  # Example user id
movie_id = 1  # Example movie id
top_similar_users = get_top_similar_users(train_sparse_matrix, user_id, movie_id, {'movie': movie_avg_ratings})
top_similar_movies = get_top_similar_movies(train_sparse_matrix, user_id, movie_id, {'user': user_avg_ratings})

# Printing features
print("Global Average Rating:", global_avg_rating)
print("Average Ratings for Users:", user_avg_ratings)
print("Average Ratings for Movies:", movie_avg_ratings)
print("Top Similar Users:", top_similar_users)
print("Top Similar Movies:", top_similar_movies)

df.head()
print(df.shape[0])

Global Average Rating: -2.45950724816013e-15
Average Ratings for Users: {0: -0.10498353298318633, 1: 0.23564869617453132, 2: 0.2004628785067822, 3: -0.7958057907796359, 4: -0.009577445962251884, 5: -0.29302466795529114, 6: 0.09956513307177049, 7: -0.0791328566213571, 8: 0.24333081040961108, 9: 0.5420337355620529, 10: -0.5807951098173831, 11: -0.16902999319502338, 12: 0.342727767263159, 13: -0.03083155459439757, 14: 0.09330597602516681, 15: 0.07962086671616302, 16: 0.22813533484377746, 17: 0.2286821430362619, 18: 0.08985798702720796, 19: 0.6336987017933434, 20: 0.6919913571198232, 21: -0.3298093212187845, 22: -0.18443290842800616, 23: -0.2883117292492355, 24: -0.24711201543449116, 25: -1.115744472476915, 26: 0.18644340330531436, 27: 0.054997522273763366, 28: -0.2800812680260482, 29: 1.1275064770654832, 30: 0.19500688233424102, 31: 0.6686502630465706, 32: -0.4272043427803226, 33: 0.2817509446520131, 34: -0.15150480366382524, 35: -0.5845948796405326, 36: -0.13149170861670942, 37: -0.15240

We calculate the global average rating using the get_global_average function defined earlier. This represents the average rating across all user-item interactions in the dataset.

We calculate average ratings for users and movies separately using the get_average_ratings function, specifying of_users=True and of_users=False respectively. These average ratings provide insights into how users and movies are rated on average.

We demonstrate an example usage of the get_top_similar_users and get_top_similar_movies functions. For a given user and movie, these functions find the top similar users and movies based on cosine similarity, respectively. We provide an example user ID and movie ID to illustrate how these functions can be used.

In [183]:
#removing rows with negative ratings
df = df[df['rating'] >= 0]

print(df.head(500))
print(df.shape[0])

         id  movieId  userId    rating
1     862.0      1.0     9.0  0.506625
2     862.0      1.0    13.0  1.615167
7     862.0      1.0    26.0  1.615167
8     862.0      1.0    30.0  0.506625
9     862.0      1.0    37.0  0.506625
...     ...      ...     ...       ...
993   524.0     16.0   433.0  1.060896
994   524.0     16.0   442.0  1.060896
995   524.0     16.0   452.0  0.506625
999   524.0     16.0   472.0  0.506625
1000  524.0     16.0   475.0  0.506625

[500 rows x 4 columns]
88912


## Modeling

## XG Boost

In [185]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import xgboost as xgb

# Splitting dataset into train and test sets
X = df.drop(['userId', 'movieId', 'rating'], axis=1)
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1: n_estimators=150, max_depth=3
xgb_model_1 = xgb.XGBRegressor(n_estimators=150, max_depth=3, random_state=42)
xgb_model_1.fit(X_train, y_train)

y_pred_train_1 = xgb_model_1.predict(X_train)
y_pred_test_1 = xgb_model_1.predict(X_test)

mae_train_1 = mean_absolute_error(y_train, y_pred_train_1)
mae_test_1 = mean_absolute_error(y_test, y_pred_test_1)

mape_train_1 = np.mean(np.abs((y_train - y_pred_train_1) / y_train)) * 100
mape_test_1 = np.mean(np.abs((y_test - y_pred_test_1) / y_test)) * 100

rmse_train_1 = np.sqrt(mean_squared_error(y_train, y_pred_train_1))
rmse_test_1 = np.sqrt(mean_squared_error(y_test, y_pred_test_1))

print("XGBoost Model (n_estimators=150, max_depth=3)")
print("Train - MAE:", mae_train_1, "MAPE:", mape_train_1, "RMSE:", rmse_train_1)
print("Test  - MAE:", mae_test_1, "MAPE:", mape_test_1, "RMSE:", rmse_test_1)




XGBoost Model (n_estimators=150, max_depth=3)
Train - MAE: 0.336508791264718 MAPE: 285338.2408639482 RMSE: 0.4430732644368297
Test  - MAE: 0.3361570454470608 MAPE: 286862.0737323651 RMSE: 0.44114297215733334


In [186]:
# Model 2: n_estimators=150, max_depth=5
xgb_model_2 = xgb.XGBRegressor(n_estimators=150, max_depth=5, random_state=42)
xgb_model_2.fit(X_train, y_train)

y_pred_train_2 = xgb_model_2.predict(X_train)
y_pred_test_2 = xgb_model_2.predict(X_test)

mae_train_2 = mean_absolute_error(y_train, y_pred_train_2)
mae_test_2 = mean_absolute_error(y_test, y_pred_test_2)

mape_train_2 = np.mean(np.abs((y_train - y_pred_train_2) / y_train)) * 100
mape_test_2 = np.mean(np.abs((y_test - y_pred_test_2) / y_test)) * 100

rmse_train_2 = np.sqrt(mean_squared_error(y_train, y_pred_train_2))
rmse_test_2 = np.sqrt(mean_squared_error(y_test, y_pred_test_2))

print("XGBoost Model (n_estimators=150, max_depth=5)")
print("Train - MAE:", mae_train_2, "MAPE:", mape_train_2, "RMSE:", rmse_train_2)
print("Test  - MAE:", mae_test_2, "MAPE:", mape_test_2, "RMSE:", rmse_test_2)




XGBoost Model (n_estimators=150, max_depth=5)
Train - MAE: 0.3359092654380769 MAPE: 283787.26350724424 RMSE: 0.44291212698394944
Test  - MAE: 0.3358800632827801 MAPE: 285730.00014035916 RMSE: 0.44133362293365663


In [187]:
# Model 3: n_estimators=200, max_depth=3
xgb_model_3 = xgb.XGBRegressor(n_estimators=200, max_depth=3, random_state=42)
xgb_model_3.fit(X_train, y_train)

y_pred_train_3 = xgb_model_3.predict(X_train)
y_pred_test_3 = xgb_model_3.predict(X_test)

mae_train_3 = mean_absolute_error(y_train, y_pred_train_3)
mae_test_3 = mean_absolute_error(y_test, y_pred_test_3)

mape_train_3 = np.mean(np.abs((y_train - y_pred_train_3) / y_train)) * 100
mape_test_3 = np.mean(np.abs((y_test - y_pred_test_3) / y_test)) * 100

rmse_train_3 = np.sqrt(mean_squared_error(y_train, y_pred_train_3))
rmse_test_3 = np.sqrt(mean_squared_error(y_test, y_pred_test_3))

print("XGBoost Model (n_estimators=200, max_depth=3)")
print("Train - MAE:", mae_train_3, "MAPE:", mape_train_3, "RMSE:", rmse_train_3)
print("Test  - MAE:", mae_test_3, "MAPE:", mape_test_3, "RMSE:", rmse_test_3)



XGBoost Model (n_estimators=200, max_depth=3)
Train - MAE: 0.33626730712173236 MAPE: 284720.3291573472 RMSE: 0.44298250425409386
Test  - MAE: 0.33605939594944667 MAPE: 286358.0195164955 RMSE: 0.4412408276871552


In [188]:
# Model 4: n_estimators=200, max_depth=5
xgb_model_4 = xgb.XGBRegressor(n_estimators=200, max_depth=5, random_state=42)
xgb_model_4.fit(X_train, y_train)

y_pred_train_4 = xgb_model_4.predict(X_train)
y_pred_test_4 = xgb_model_4.predict(X_test)

mae_train_4 = mean_absolute_error(y_train, y_pred_train_4)
mae_test_4 = mean_absolute_error(y_test, y_pred_test_4)

mape_train_4 = np.mean(np.abs((y_train - y_pred_train_4) / y_train)) * 100
mape_test_4 = np.mean(np.abs((y_test - y_pred_test_4) / y_test)) * 100

rmse_train_4 = np.sqrt(mean_squared_error(y_train, y_pred_train_4))
rmse_test_4 = np.sqrt(mean_squared_error(y_test, y_pred_test_4))

print("XGBoost Model (n_estimators=200, max_depth=5)")
print("Train - MAE:", mae_train_4, "MAPE:", mape_train_4, "RMSE:", rmse_train_4)
print("Test  - MAE:", mae_test_4, "MAPE:", mape_test_4, "RMSE:", rmse_test_4)



XGBoost Model (n_estimators=200, max_depth=5)
Train - MAE: 0.3358532817633881 MAPE: 283655.9190850722 RMSE: 0.44290920698603586
Test  - MAE: 0.3358496358717816 MAPE: 285651.1410072271 RMSE: 0.4413484019488108


In [189]:
# Model 5: n_estimators=250, max_depth=3
xgb_model_5 = xgb.XGBRegressor(n_estimators=250, max_depth=3, random_state=42)
xgb_model_5.fit(X_train, y_train)

y_pred_train_5 = xgb_model_5.predict(X_train)
y_pred_test_5 = xgb_model_5.predict(X_test)

mae_train_5 = mean_absolute_error(y_train, y_pred_train_5)
mae_test_5 = mean_absolute_error(y_test, y_pred_test_5)

mape_train_5 = np.mean(np.abs((y_train - y_pred_train_5) / y_train)) * 100
mape_test_5 = np.mean(np.abs((y_test - y_pred_test_5) / y_test)) * 100

rmse_train_5 = np.sqrt(mean_squared_error(y_train, y_pred_train_5))
rmse_test_5 = np.sqrt(mean_squared_error(y_test, y_pred_test_5))

print("XGBoost Model (n_estimators=250, max_depth=3)")
print("Train - MAE:", mae_train_5, "MAPE:", mape_train_5, "RMSE:", rmse_train_5)
print("Test  - MAE:", mae_test_5, "MAPE:", mape_test_5, "RMSE:", rmse_test_5)



XGBoost Model (n_estimators=250, max_depth=3)
Train - MAE: 0.33610795255201303 MAPE: 284280.39527147415 RMSE: 0.4429420563193425
Test  - MAE: 0.33598568632300063 MAPE: 286071.5535743935 RMSE: 0.4412786205981584


In [190]:
# Model 6: n_estimators=250, max_depth=5
xgb_model_6 = xgb.XGBRegressor(n_estimators=250, max_depth=5, random_state=42)
xgb_model_6.fit(X_train, y_train)

y_pred_train_6 = xgb_model_6.predict(X_train)
y_pred_test_6 = xgb_model_6.predict(X_test)

mae_train_6 = mean_absolute_error(y_train, y_pred_train_6)
mae_test_6 = mean_absolute_error(y_test, y_pred_test_6)

mape_train_6 = np.mean(np.abs((y_train - y_pred_train_6) / y_train)) * 100
mape_test_6 = np.mean(np.abs((y_test - y_pred_test_6) / y_test)) * 100

rmse_train_6 = np.sqrt(mean_squared_error(y_train, y_pred_train_6))
rmse_test_6 = np.sqrt(mean_squared_error(y_test, y_pred_test_6))

print("XGBoost Model (n_estimators=250, max_depth=5)")
print("Train - MAE:", mae_train_6, "MAPE:", mape_train_6, "RMSE:", rmse_train_6)
print("Test  - MAE:", mae_test_6, "MAPE:", mape_test_6, "RMSE:", rmse_test_6)

XGBoost Model (n_estimators=250, max_depth=5)
Train - MAE: 0.33583488815607954 MAPE: 283604.36593614734 RMSE: 0.4429087474273247
Test  - MAE: 0.33584104859131897 MAPE: 285616.5432236419 RMSE: 0.4413560444323549


## Random Forest

In [192]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Model 1: n_estimators=150, max_depth=3
rf_model_1 = RandomForestRegressor(n_estimators=150, max_depth=3, random_state=42)
rf_model_1.fit(X_train, y_train)

y_pred_rf_train_1 = rf_model_1.predict(X_train)
y_pred_rf_test_1 = rf_model_1.predict(X_test)

mae_rf_train_1 = mean_absolute_error(y_train, y_pred_rf_train_1)
mae_rf_test_1 = mean_absolute_error(y_test, y_pred_rf_test_1)

mape_rf_train_1 = np.mean(np.abs((y_train - y_pred_rf_train_1) / y_train)) * 100
mape_rf_test_1 = np.mean(np.abs((y_test - y_pred_rf_test_1) / y_test)) * 100

rmse_rf_train_1 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_1))
rmse_rf_test_1 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_1))

print("Random Forest Model (n_estimators=150, max_depth=3)")
print("Train - MAE:", mae_rf_train_1, "MAPE:", mape_rf_train_1, "RMSE:", rmse_rf_train_1)
print("Test  - MAE:", mae_rf_test_1, "MAPE:", mape_rf_test_1, "RMSE:", rmse_rf_test_1)



Random Forest Model (n_estimators=150, max_depth=3)
Train - MAE: 0.3419185387845838 MAPE: 294156.64104368933 RMSE: 0.44714069667913103
Test  - MAE: 0.34043636254391146 MAPE: 294609.7616678047 RMSE: 0.44389761460631055


In [193]:
# Model 2: n_estimators=150, max_depth=5
rf_model_2 = RandomForestRegressor(n_estimators=150, max_depth=5, random_state=42)
rf_model_2.fit(X_train, y_train)

y_pred_rf_train_2 = rf_model_2.predict(X_train)
y_pred_rf_test_2 = rf_model_2.predict(X_test)

mae_rf_train_2 = mean_absolute_error(y_train, y_pred_rf_train_2)
mae_rf_test_2 = mean_absolute_error(y_test, y_pred_rf_test_2)

mape_rf_train_2 = np.mean(np.abs((y_train - y_pred_rf_train_2) / y_train)) * 100
mape_rf_test_2 = np.mean(np.abs((y_test - y_pred_rf_test_2) / y_test)) * 100

rmse_rf_train_2 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_2))
rmse_rf_test_2 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_2))

print("Random Forest Model (n_estimators=150, max_depth=5)")
print("Train - MAE:", mae_rf_train_2, "MAPE:", mape_rf_train_2, "RMSE:", rmse_rf_train_2)
print("Test  - MAE:", mae_rf_test_2, "MAPE:", mape_rf_test_2, "RMSE:", rmse_rf_test_2)



Random Forest Model (n_estimators=150, max_depth=5)
Train - MAE: 0.33724228395902944 MAPE: 283165.5907909499 RMSE: 0.44276291557302544
Test  - MAE: 0.3358428171522024 MAPE: 283815.4934617922 RMSE: 0.4397289869406362


In [194]:
# Model 3: n_estimators=200, max_depth=3
rf_model_3 = RandomForestRegressor(n_estimators=200, max_depth=3, random_state=42)
rf_model_3.fit(X_train, y_train)

y_pred_rf_train_3 = rf_model_3.predict(X_train)
y_pred_rf_test_3 = rf_model_3.predict(X_test)

mae_rf_train_3 = mean_absolute_error(y_train, y_pred_rf_train_3)
mae_rf_test_3 = mean_absolute_error(y_test, y_pred_rf_test_3)

mape_rf_train_3 = np.mean(np.abs((y_train - y_pred_rf_train_3) / y_train)) * 100
mape_rf_test_3 = np.mean(np.abs((y_test - y_pred_rf_test_3) / y_test)) * 100

rmse_rf_train_3 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_3))
rmse_rf_test_3 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_3))

print("Random Forest Model (n_estimators=200, max_depth=3)")
print("Train - MAE:", mae_rf_train_3, "MAPE:", mape_rf_train_3, "RMSE:", rmse_rf_train_3)
print("Test  - MAE:", mae_rf_test_3, "MAPE:", mape_rf_test_3, "RMSE:", rmse_rf_test_3)



Random Forest Model (n_estimators=200, max_depth=3)
Train - MAE: 0.34191514996726613 MAPE: 294185.27620392514 RMSE: 0.4471495810767071
Test  - MAE: 0.3404292900090851 MAPE: 294633.4628808787 RMSE: 0.44390793829192843


In [195]:
# Model 4: n_estimators=200, max_depth=5
rf_model_4 = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)
rf_model_4.fit(X_train, y_train)

y_pred_rf_train_4 = rf_model_4.predict(X_train)
y_pred_rf_test_4 = rf_model_4.predict(X_test)

mae_rf_train_4 = mean_absolute_error(y_train, y_pred_rf_train_4)
mae_rf_test_4 = mean_absolute_error(y_test, y_pred_rf_test_4)

mape_rf_train_4 = np.mean(np.abs((y_train - y_pred_rf_train_4) / y_train)) * 100
mape_rf_test_4 = np.mean(np.abs((y_test - y_pred_rf_test_4) / y_test)) * 100

rmse_rf_train_4 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_4))
rmse_rf_test_4 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_4))

print("Random Forest Model (n_estimators=200, max_depth=5)")
print("Train - MAE:", mae_rf_train_4, "MAPE:", mape_rf_train_4, "RMSE:", rmse_rf_train_4)
print("Test  - MAE:", mae_rf_test_4, "MAPE:", mape_rf_test_4, "RMSE:", rmse_rf_test_4)



Random Forest Model (n_estimators=200, max_depth=5)
Train - MAE: 0.3372377354135436 MAPE: 283236.8330677261 RMSE: 0.4427606052030974
Test  - MAE: 0.33582514447524553 MAPE: 283852.35914841207 RMSE: 0.43971320268627323


In [196]:
# Model 5: n_estimators=250, max_depth=3
rf_model_5 = RandomForestRegressor(n_estimators=250, max_depth=3, random_state=42)
rf_model_5.fit(X_train, y_train)

y_pred_rf_train_5 = rf_model_5.predict(X_train)
y_pred_rf_test_5 = rf_model_5.predict(X_test)

mae_rf_train_5 = mean_absolute_error(y_train, y_pred_rf_train_5)
mae_rf_test_5 = mean_absolute_error(y_test, y_pred_rf_test_5)

mape_rf_train_5 = np.mean(np.abs((y_train - y_pred_rf_train_5) / y_train)) * 100
mape_rf_test_5 = np.mean(np.abs((y_test - y_pred_rf_test_5) / y_test)) * 100

rmse_rf_train_5 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_5))
rmse_rf_test_5 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_5))

print("Random Forest Model (n_estimators=250, max_depth=3)")
print("Train -MAE:", mae_rf_train_5, "MAPE:", mape_rf_train_5, "RMSE:", rmse_rf_train_5)
print("Test  -MAE:", mae_rf_test_5, "MAPE:", mape_rf_test_5, "RMSE:", rmse_rf_test_5)


Random Forest Model (n_estimators=250, max_depth=3)
Train -MAE: 0.3419663995044627 MAPE: 294288.97120371787 RMSE: 0.44716306876317224
Test  -MAE: 0.3404829855948533 MAPE: 294743.88236868114 RMSE: 0.4439218831673507


In [197]:
# Model 6: n_estimators=250, max_depth=5
rf_model_6 = RandomForestRegressor(n_estimators=250, max_depth=5, random_state=42)
rf_model_6.fit(X_train, y_train)

y_pred_rf_train_6 = rf_model_6.predict(X_train)
y_pred_rf_test_6 = rf_model_6.predict(X_test)

mae_rf_train_6 = mean_absolute_error(y_train, y_pred_rf_train_6)
mae_rf_test_6 = mean_absolute_error(y_test, y_pred_rf_test_6)

mape_rf_train_6 = np.mean(np.abs((y_train - y_pred_rf_train_6) / y_train)) * 100
mape_rf_test_6 = np.mean(np.abs((y_test - y_pred_rf_test_6) / y_test)) * 100

rmse_rf_train_6 = np.sqrt(mean_squared_error(y_train, y_pred_rf_train_6))
rmse_rf_test_6 = np.sqrt(mean_squared_error(y_test, y_pred_rf_test_6))

print("Random Forest Model (n_estimators=250, max_depth=5)")
print("Train - MAE:", mae_rf_train_6, "MAPE:", mape_rf_train_6, "RMSE:", rmse_rf_train_6)
print("Test  - MAE:", mae_rf_test_6, "MAPE:", mape_rf_test_6, "RMSE:", rmse_rf_test_6)



Random Forest Model (n_estimators=250, max_depth=5)
Train - MAE: 0.3372741194446346 MAPE: 283352.66762217414 RMSE: 0.44277008781210153
Test  - MAE: 0.33583948767443517 MAPE: 283971.984050414 RMSE: 0.43967896031028947


Here's an interpretation of the performance:

XGBoost Model:
Across all configurations, the training RMSE values are consistently low, ranging from approximately 0.443 to 0.443. Lower RMSE values indicate better model performance in terms of how closely the predicted ratings align with the actual ratings in the training data.
There's no significant difference in RMSE between different combinations of n_estimators and max_depth, suggesting that the performance is relatively stable across these parameters.

Random Forest Model:
Similar to XGBoost, the Random Forest model also exhibits low training RMSE values, ranging from approximately 0.442 to 0.447.
The Random Forest model with max_depth=5 generally performs slightly better than the one with max_depth=3, as evidenced by the consistently lower RMSE values across different numbers of estimators (n_estimators).

In conclusion, both models seem to capture the patterns in the training data quite well, as indicated by the low training RMSE values.